<a href="https://colab.research.google.com/github/venkat23-hub/VenkatKWT/blob/main/KWT_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install torch torchaudio librosa numpy tqdm pyyaml


  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
Using cached nvidia_cublas_

**To check dataset is present or not.**

In [ ]:
import os

# Define dataset path
DATASET_PATH = "/content/drive/MyDrive/Keyword_spotting_transformer/speech_commands_dataset"

# Check if dataset exists
if os.path.exists(DATASET_PATH):
    print("✅ Dataset found!")
else:
    print("❌ Dataset not found. Check the path!")


✅ Dataset found!


**Organize Train, Validation, and Test Splits**

In [ ]:
!pip install --upgrade numba librosa

**Verify Processed data**

In [ ]:
import os

# Check if preprocessing was successful
print("Train files:", len(os.listdir("/content/drive/MyDrive/Keyword_spotting_transformer/data/train/")))
print("Validation files:", len(os.listdir("/content/drive/MyDrive/Keyword_spotting_transformer/data/valid")))
print("Test files:", len(os.listdir("/content/drive/MyDrive/Keyword_spotting_transformer/data/test")))


Train files: 30
Validation files: 30
Test files: 30


**Converting the .wav files to MEL spectrograms**

In [ ]:
import os

DATA_PATH = "/content/drive/MyDrive/Keyword_spotting_transformer/data"

print("Train folders:", os.listdir(f"{DATA_PATH}/train"))
print("Valid folders:", os.listdir(f"{DATA_PATH}/valid"))
print("Test folders:", os.listdir(f"{DATA_PATH}/test"))


Train folders: ['bed', 'bird', 'cat', 'dog', 'down', 'eight', 'five', 'four', 'go', 'happy', 'house', 'left', 'marvin', 'nine', 'no', 'off', 'on', 'one', 'right', 'seven', 'sheila', 'six', 'stop', 'three', 'tree', 'two', 'up', 'wow', 'yes', 'zero']
Valid folders: ['bed', 'bird', 'cat', 'dog', 'down', 'eight', 'five', 'four', 'go', 'happy', 'house', 'left', 'marvin', 'nine', 'no', 'off', 'on', 'one', 'right', 'seven', 'sheila', 'six', 'stop', 'three', 'tree', 'two', 'up', 'wow', 'yes', 'zero']
Test folders: ['bed', 'bird', 'cat', 'dog', 'down', 'eight', 'five', 'four', 'go', 'happy', 'house', 'left', 'marvin', 'nine', 'no', 'off', 'on', 'one', 'right', 'seven', 'sheila', 'six', 'stop', 'three', 'tree', 'two', 'up', 'wow', 'yes', 'zero']


In [ ]:
import os
import shutil

TRAIN_PATH = "/content/drive/MyDrive/Keyword_spotting_transformer/data/train"
PROCESSED_PATH = "/content/drive/MyDrive/Keyword_spotting_transformer/data/train_processed"

os.makedirs(PROCESSED_PATH, exist_ok=True)

for file in os.listdir(TRAIN_PATH):
    if file.endswith(".npy"):
        shutil.move(os.path.join(TRAIN_PATH, file), os.path.join(PROCESSED_PATH, file))

print("✅ Moved .npy files to train_processed/")


✅ Moved .npy files to train_processed/


In [ ]:
import os
import shutil

DATA_PATH = "/content/drive/MyDrive/Keyword_spotting_transformer/data"
SPLITS = ["valid", "test"]

for split in SPLITS:
    input_folder = os.path.join(DATA_PATH, split)
    output_folder = os.path.join(DATA_PATH, f"{split}_processed")

    os.makedirs(output_folder, exist_ok=True)

    for file in os.listdir(input_folder):
        if file.endswith(".npy"):
            shutil.move(os.path.join(input_folder, file), os.path.join(output_folder, file))

    print(f" Moved .npy files from {split} → {split}_processed/")


✅ Moved .npy files from valid → valid_processed/
✅ Moved .npy files from test → test_processed/


In [ ]:
import os

DATA_PATH = "/content/drive/MyDrive/Keyword_spotting_transformer/data"

print("Train Processed:", len(os.listdir(f"{DATA_PATH}/train_processed")))
print("Valid Processed:", len(os.listdir(f"{DATA_PATH}/valid_processed")))
print("Test Processed:", len(os.listdir(f"{DATA_PATH}/test_processed")))


Train Processed: 51324
Valid Processed: 6798
Test Processed: 6835


In [ ]:
import os
import torch
import torchaudio
import torchaudio.transforms as T
import numpy as np
from tqdm import tqdm

# Parameters from the research paper
SAMPLE_RATE = 16000
N_MELS = 40
N_FFT = 400  # Window size (30ms)
HOP_LENGTH = 160  # Hop size (10ms)

# MelSpectrogram transformation using torchaudio
mel_transform = T.MelSpectrogram(
    sample_rate=SAMPLE_RATE,
    n_fft=N_FFT,
    hop_length=HOP_LENGTH,
    n_mels=N_MELS,
    f_min=0.0,
    f_max=None,
    power=2.0,
)

# Paths
DATA_PATH = "/content/drive/MyDrive/Keyword_spotting_transformer/data"
DATA_SPLITS = ["train", "valid", "test"]

# Function to process audio into Mel-Spectrogram using torchaudio
def process_audio(file_path):
    waveform, sr = torchaudio.load(file_path)  # Load audio
    if sr != SAMPLE_RATE:
        waveform = torchaudio.functional.resample(waveform, sr, SAMPLE_RATE)  # Resample if needed
    mel_spec = mel_transform(waveform).squeeze(0)  # Convert to Mel Spectrogram and remove extra channel
    mel_spec_db = torch.log(mel_spec + 1e-6).numpy()  # Convert to log scale (avoid log(0) errors)
    return mel_spec_db

# Iterate over splits and process files
for split in DATA_SPLITS:
    input_folder = os.path.join(DATA_PATH, split)
    output_folder = os.path.join(DATA_PATH, f"{split}_processed")
    os.makedirs(output_folder, exist_ok=True)

    print(f"Processing {split} data...")

    for keyword in os.listdir(input_folder):
        keyword_path = os.path.join(input_folder, keyword)
        if not os.path.isdir(keyword_path):
            continue  # Skip non-folder files

        for filename in tqdm(os.listdir(keyword_path), desc=f"Processing {keyword}"):
            file_path = os.path.join(keyword_path, filename)
            if not file_path.endswith(".wav"):
                continue  # Skip non-audio files

            try:
                mel_spec = process_audio(file_path)
                save_filename = f"{keyword}_{filename.replace('.wav', '.npy')}"  # Add keyword to filename
                save_path = os.path.join(output_folder, save_filename)

                np.save(save_path, mel_spec)
               # print(f"✅ Processed: {file_path} → {save_path}")  # Debugging print

            except Exception as e:
                print(f"❌ Error processing {file_path}: {e}")

print("✅ Preprocessing complete!")


Processing train data...


Processing zero: 100%|██████████| 1866/1866 [01:50<00:00, 16.89it/s]


Processing valid data...


Processing zero: 100%|██████████| 260/260 [00:11<00:00, 23.30it/s]


Processing test data...


Processing zero: 100%|██████████| 250/250 [00:18<00:00, 13.85it/s]

✅ Preprocessing complete!


**Verifying the spectrograms**

In [ ]:
import os

data_path = "/content/drive/MyDrive/Keyword_spotting_transformer/data"

print("Train Spectrograms:", len(os.listdir(f"{data_path}/train_processed")))
print("Validation Spectrograms:", len(os.listdir(f"{data_path}/valid_processed")))
print("Test Spectrograms:", len(os.listdir(f"{data_path}/test_processed")))


Train Spectrograms: 51318
Validation Spectrograms: 6798
Test Spectrograms: 6835


**To patch the spectrograms**

In [ ]:
import os
import numpy as np

DATA_PATH = "/content/drive/MyDrive/Keyword_spotting_transformer/data/train_processed"

labels = set()
for file in os.listdir(DATA_PATH):
    if file.endswith(".npy"):
        label = file.split("_")[0]  # Extracts the word label from filename
        labels.add(label)

print("Available Labels:", labels)
print("Total Labels:", len(labels))


Available Labels: {'two', 'tree', 'nine', 'six', 'right', 'three', 'no', 'yes', 'one', 'stop', 'wow', 'off', 'sheila', 'bed', 'go', 'seven', 'cat', 'four', 'on', 'eight', 'bird', 'down', 'marvin', 'house', 'happy', 'dog', 'five', 'left', 'zero', 'up'}
Total Labels: 30


In [ ]:
import os
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader

# Define the fixed spectrogram size (mel-bands x time-frames)
N_MELS = 40  # Mel-frequency bands (height)
FIXED_TIME_DIM = 101  # Fixed width for spectrograms (adjust based on dataset)

# Function to pad or crop spectrograms to a fixed size
def pad_or_crop_spectrogram(spectrogram, fixed_size=FIXED_TIME_DIM):
    """Pads with zeros or crops spectrogram to a fixed time size"""
    time_dim = spectrogram.shape[1]  # Get the time dimension

    if time_dim < fixed_size:
        # Pad with zeros if too short
        pad_width = fixed_size - time_dim
        spectrogram = np.pad(spectrogram, ((0, 0), (0, pad_width)), mode="constant")
    elif time_dim > fixed_size:
        # Crop if too long
        spectrogram = spectrogram[:, :fixed_size]

    return spectrogram

# Custom Dataset class for Keyword Spotting
class KeywordDataset(Dataset):
    def __init__(self, spectrogram_dir, label_dict):
        """
        spectrogram_dir: Path to processed spectrogram .npy files
        label_dict: Dictionary mapping word labels to numerical values
        """
        self.spectrogram_files = []
        self.labels = []

        # Scan the directory for spectrograms
        for filename in os.listdir(spectrogram_dir):
            if filename.endswith(".npy"):
                word = filename.split("_")[0]  # Extract the keyword label
                if word in label_dict:
                    self.spectrogram_files.append(os.path.join(spectrogram_dir, filename))
                    self.labels.append(label_dict[word])

    def __len__(self):
        return len(self.spectrogram_files)

    def __getitem__(self, idx):
        # Load spectrogram
        spectrogram = np.load(self.spectrogram_files[idx])
        spectrogram = pad_or_crop_spectrogram(spectrogram)  # Ensure fixed size
        spectrogram = torch.tensor(spectrogram, dtype=torch.float32)

        # Load label
        label = self.labels[idx]
        return spectrogram, label

# Define label dictionary (ensure it matches your selected classes)
label_dict = {
    'bed':0, 'bird':1, 'cat':2, 'dog':3, 'down':4,
    'eight':5, 'five':6, 'four':7, 'go':8, 'happy':9,
    'house':10, 'left':11, 'marvin':12, 'nine':13, 'no':14,
    'off':15, 'on':16, 'one':17, 'right':18, 'seven':19,
    'sheila':20, 'six':21, 'stop':22, 'three':23, 'tree':24,
    'two':25, 'up':26, 'wow':27, 'yes':28,'zero':29
}

# Paths to preprocessed spectrograms
TRAIN_DIR = "/content/drive/MyDrive/Keyword_spotting_transformer/data/train_processed"
VALID_DIR = "/content/drive/MyDrive/Keyword_spotting_transformer/data/valid_processed"
TEST_DIR = "/content/drive/MyDrive/Keyword_spotting_transformer/data/test_processed"

# Create datasets
train_dataset = KeywordDataset(TRAIN_DIR, label_dict)
valid_dataset = KeywordDataset(VALID_DIR, label_dict)
test_dataset = KeywordDataset(TEST_DIR, label_dict)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

# Test DataLoader
for spectrograms, labels in train_loader:
    print("Spectrogram Shape:", spectrograms.shape)  # Expected: (batch_size, 40, 101)
    print("Labels:", labels)
    break  # Only print one batch


Spectrogram Shape: torch.Size([32, 40, 101])
Labels: tensor([18,  5, 19, 24, 28,  6, 14, 22, 17, 27, 14, 10, 18, 28, 29, 23,  4,  6,
         0, 19,  3,  2,  6, 29, 26, 25, 15, 14, 22,  8, 12, 19])


In [ ]:
# Try loading a single batch
data_iter = iter(train_loader)
try:
    spectrograms, labels = next(data_iter)
    print(" Successfully loaded a batch!")
    print("Spectrogram Shape:", spectrograms.shape)
    print("Labels:", labels)
except Exception as e:
    print(" Error in DataLoader:", e)


✅ Successfully loaded a batch!
Spectrogram Shape: torch.Size([32, 40, 101])
Labels: tensor([28,  0, 26,  5, 20,  8, 24,  4, 13, 15,  7, 22, 11,  3, 25, 16, 22,  2,
        29, 23, 19, 24, 13, 14,  0, 20,  8,  2, 10, 28, 13, 11])


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# === Hyperparameters ===
PATCH_SIZE = 16  # Patch size along time axis
EMBED_DIM = 128  # Embedding dimension for transformer input
N_MELS = 40      # Mel spectrogram height

# === 1. Function to Split into Patches ===
def split_into_patches(spectrograms, patch_size=PATCH_SIZE):
    """
    Splits spectrograms into non-overlapping patches along the time axis.

    Args:
        spectrograms (torch.Tensor): (batch_size, n_mels, time_frames)
        patch_size (int): Size of each patch along the time axis.

    Returns:
        torch.Tensor: (batch_size, num_patches, n_mels * patch_size)
    """
    batch_size, n_mels, time_frames = spectrograms.shape

    # Pad time axis if it's not divisible by patch_size
    pad_len = (patch_size - (time_frames % patch_size)) % patch_size
    spectrograms_padded = F.pad(spectrograms, (0, pad_len))  # Pad along time axis

    # Reshape into patches: (batch, n_mels, num_patches, patch_size)
    num_patches = spectrograms_padded.shape[-1] // patch_size
    patches = spectrograms_padded.unfold(dimension=2, size=patch_size, step=patch_size)

    # Flatten patches: (batch, num_patches, n_mels * patch_size)
    patches = patches.permute(0, 2, 1, 3).reshape(batch_size, num_patches, -1)

    return patches

# === 2. Linear Projection Layer ===
class PatchEmbedding(nn.Module):
    """
    Projects patches into an embedding space and adds positional embeddings.
    """
    def __init__(self, patch_dim, embed_dim, max_patches=64):
        super().__init__()
        self.projection = nn.Linear(patch_dim, embed_dim)  # Linear mapping
        self.positional_embedding = nn.Parameter(torch.randn(1, max_patches, embed_dim))  # Learnable position embeddings

    def forward(self, x):
        """
        x: (batch_size, num_patches, patch_dim)
        Returns: (batch_size, num_patches, embed_dim)
        """
        x = self.projection(x)  # Apply linear projection
        x += self.positional_embedding[:, :x.shape[1], :]  # Add positional embedding
        return x

# === Example Usage ===
batch_size = 32
time_frames = 101  # Example time dimension

# Create dummy spectrogram batch (32, 40, 101)
dummy_spectrograms = torch.randn(batch_size, N_MELS, time_frames)

# Split into patches
patched_spectrograms = split_into_patches(dummy_spectrograms, PATCH_SIZE)

# Initialize Patch Embedding layer
patch_embedding_layer = PatchEmbedding(N_MELS * PATCH_SIZE, EMBED_DIM)

# Apply Patch Embedding Layer
embedded_patches = patch_embedding_layer(patched_spectrograms)

print("Patched Shape:", patched_spectrograms.shape)  # (32, num_patches, patch_size * 40)
print("Embedded Shape:", embedded_patches.shape)  # (32, num_patches, EMBED_DIM)


Patched Shape: torch.Size([32, 7, 640])
Embedded Shape: torch.Size([32, 7, 128])


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TransformerEncoderBlock(nn.Module):
    """
    A single Transformer encoder block with MHSA, LayerNorm, and FeedForward layers.
    """
    def __init__(self, embed_dim, num_heads, expansion_factor=4, dropout=0.1):
        super().__init__()
        self.attention = nn.MultiheadAttention(embed_dim, num_heads, dropout=dropout, batch_first=True)
        self.norm1 = nn.LayerNorm(embed_dim)
        self.ffn = nn.Sequential(
            nn.Linear(embed_dim, embed_dim * expansion_factor),
            nn.GELU(),
            nn.Linear(embed_dim * expansion_factor, embed_dim),
        )
        self.norm2 = nn.LayerNorm(embed_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        """
        Forward pass through a Transformer encoder block.
        Args:
            x: (batch_size, num_patches, embed_dim)
        Returns:
            x: (batch_size, num_patches, embed_dim)
        """
        # Multi-Head Self Attention
        attn_output, _ = self.attention(x, x, x)
        x = self.norm1(x + self.dropout(attn_output))  # Add & Norm

        # Feed Forward Network
        ffn_output = self.ffn(x)
        x = self.norm2(x + self.dropout(ffn_output))  # Add & Norm

        return x

class KeywordTransformer(nn.Module):
    """
    Keyword Transformer (KWT) model for keyword spotting.
    """
    def __init__(self, embed_dim=128, num_heads=4, num_layers=6, num_classes=10, patch_dim=640, max_patches=64, dropout=0.1):
        super().__init__()

        # Patch Embedding
        self.patch_embedding = nn.Linear(patch_dim, embed_dim)
        self.positional_embedding = nn.Parameter(torch.randn(1, max_patches, embed_dim))

        # Transformer Encoder
        self.encoder_layers = nn.ModuleList([
            TransformerEncoderBlock(embed_dim, num_heads, dropout=dropout) for _ in range(num_layers)
        ])

        # Classification Head
        self.cls_token = nn.Parameter(torch.randn(1, 1, embed_dim))
        self.mlp_head = nn.Sequential(
            nn.LayerNorm(embed_dim),
            nn.Linear(embed_dim, num_classes)
        )

    def forward(self, x):
        """
        Forward pass through KWT model.
        Args:
            x: (batch_size, num_patches, patch_dim)
        Returns:
            logits: (batch_size, num_classes)
        """
        batch_size = x.shape[0]
        x = self.patch_embedding(x)  # Project patches to embedding space
        x += self.positional_embedding[:, :x.shape[1], :]  # Add positional encoding

        # Add CLS Token
        cls_tokens = self.cls_token.expand(batch_size, -1, -1)
        x = torch.cat([cls_tokens, x], dim=1)

        # Pass through Transformer Encoder
        for layer in self.encoder_layers:
            x = layer(x)

        # Classification: Use CLS token output
        cls_output = x[:, 0, :]
        logits = self.mlp_head(cls_output)

        return logits

# === Example Usage ===
batch_size = 32
num_patches = 6  # Example number of patches
patch_dim = 640  # 40x16 flattened

dummy_patches = torch.randn(batch_size, num_patches, patch_dim)  # Fake input data
kwt_model = KeywordTransformer(embed_dim=128, num_heads=4, num_layers=6, num_classes=10)

logits = kwt_model(dummy_patches)  # Forward pass
print("Output Shape:", logits.shape)  # Expected: (batch_size, num_classes)


Output Shape: torch.Size([32, 10])


In [ ]:
import os
import torch
import numpy as np
from torch.utils.data import Dataset

class KeywordDataset(Dataset):
    """
    PyTorch Dataset to load spectrogram patches from train_processed, valid_processed, and test_processed.
    """
    def __init__(self, spectrogram_dir, label_dict):
        """
        Args:
            spectrogram_dir: Path to folder containing .npy spectrograms.
            label_dict: Dictionary mapping keywords to label indices.
        """
        self.spectrogram_files = []
        self.labels = []

        # Check if the directory exists and has files
        if not os.path.exists(spectrogram_dir):
            raise FileNotFoundError(f"Directory not found: {spectrogram_dir}")
        if not os.listdir(spectrogram_dir):
             raise FileNotFoundError(f"Directory is empty: {spectrogram_dir}")


        # Scan the directory and collect spectrogram file paths and labels
        for filename in os.listdir(spectrogram_dir):
            if filename.endswith(".npy"):
                word = filename.split("_")[0]  # Extract keyword from filename
                if word in label_dict:
                    self.spectrogram_files.append(os.path.join(spectrogram_dir, filename))
                    self.labels.append(label_dict[word])  # Convert word to numerical label

    def __len__(self):
        return len(self.spectrogram_files)

    def __getitem__(self, idx):
        """
        Loads and returns a spectrogram tensor and its corresponding label.
        """
        spectrogram = np.load(self.spectrogram_files[idx])  # Load spectrogram
        spectrogram = torch.tensor(spectrogram, dtype=torch.float32)  # Convert to tensor
        label = self.labels[idx]
        return spectrogram, label  # Return spectrogram & label tensors


# Define label mapping
label_dict = {
   'bed':0, 'bird':1, 'cat':2, 'dog':3, 'down':4,
    'eight':5, 'five':6, 'four':7, 'go':8, 'happy':9,
    'house':10, 'left':11, 'marvin':12, 'nine':13, 'no':14,
    'off':15, 'on':16, 'one':17, 'right':18, 'seven':19,
    'sheila':20, 'six':21, 'stop':22, 'three':23, 'tree':24,
    'two':25, 'up':26, 'wow':27, 'yes':28,'zero':29
}

# Define dataset paths
TRAIN_DIR = "/content/drive/MyDrive/Keyword_spotting_transformer/data/train_processed"
VALID_DIR = "/content/drive/MyDrive/Keyword_spotting_transformer/data/valid_processed"

# Create dataset instances
try:
    print(f"Checking {TRAIN_DIR}")
    train_dataset = KeywordDataset(TRAIN_DIR, label_dict)
    print(f"Number of files in training dataset: {len(train_dataset)}")
except Exception as e:
    print(f"Error creating training dataset: {e}")
    raise
try:
    print(f"Checking {VALID_DIR}")
    valid_dataset = KeywordDataset(VALID_DIR, label_dict)
    print(f"Number of files in validation dataset: {len(valid_dataset)}")
except Exception as e:
    print(f"Error creating validation dataset: {e}")
    raise


# Create DataLoaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=32, shuffle=False)

Checking /content/drive/MyDrive/Keyword_spotting_transformer/data/train_processed
Number of files in training dataset: 51318
Checking /content/drive/MyDrive/Keyword_spotting_transformer/data/valid_processed
Number of files in validation dataset: 6798


In [ ]:
import torch
from torch import nn
from einops import rearrange, repeat
from einops.layers.torch import Rearrange
import torch.nn.functional as F


class PreNorm(nn.Module):
    def __init__(self, dim, fn):
        super().__init__()
        self.norm = nn.LayerNorm(dim)
        self.fn = fn
    def forward(self, x, **kwargs):
        return self.fn(self.norm(x), **kwargs)

class FeedForward(nn.Module):
    def __init__(self, dim, hidden_dim, dropout = 0.):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, dim),
            nn.Dropout(dropout)
        )
    def forward(self, x):
        return self.net(x)

class Attention(nn.Module):
    def __init__(self, dim, heads = 8, dim_head = 64, dropout = 0.):
        super().__init__()
        inner_dim = dim_head * heads
        self.heads = heads
        self.scale = dim_head ** -0.5

        self.attend = nn.Softmax(dim=-1)
        self.to_qkv = nn.Linear(dim, inner_dim * 3, bias=False)
        self.to_out = nn.Sequential(nn.Linear(inner_dim, dim), nn.Dropout(dropout))

    def forward(self, x):
        b, n, _, h = *x.shape, self.heads
        qkv = self.to_qkv(x).chunk(3, dim=-1)
        q, k, v = map(lambda t: rearrange(t, "b n (h d) -> b h n d", h=h), qkv)

        dots = torch.einsum("b h i d, b h j d -> b h i j", q, k) * self.scale
        attn = self.attend(dots)

        out = torch.einsum("b h i j, b h j d -> b h i d", attn, v)
        out = rearrange(out, "b h n d -> b n (h d)")
        return self.to_out(out)

class Transformer(nn.Module):
    def __init__(self, dim, depth, heads, dim_head, mlp_dim, dropout=0.):
        super().__init__()
        self.layers = nn.ModuleList([
            nn.ModuleList([
                PreNorm(dim, Attention(dim, heads=heads, dim_head=dim_head, dropout=dropout)),
                PreNorm(dim, FeedForward(dim, mlp_dim, dropout=dropout))
            ]) for _ in range(depth)
        ])

    def forward(self, x):
        for attn, ff in self.layers:
            x = attn(x) + x
            x = ff(x) + x
        return x

class KeywordTransformer(nn.Module):
    def __init__(self, *, spectrogram_size, patch_size, num_classes, dim, depth, heads, mlp_dim, dropout=0.1, emb_dropout=0.1):
        super().__init__()
        self.spectrogram_size = spectrogram_size
        self.patch_size = patch_size

        # Calculate the number of patches based on the *padded* size
        max_time_frames = spectrogram_size[1]

        # Calculate the maximum possible number of patches
        num_patches = (max_time_frames + (patch_size[1] - 1)) // patch_size[1] # ceiling division

        patch_dim = spectrogram_size[0] * patch_size[1]  # (40 * patch_size)

        self.to_patch_embedding = nn.Sequential(
            # Rearrange('b c f (t p2) -> b t (c f p2)', p2=patch_size[1]),  # Keep 'c' in the output
            nn.Linear(patch_dim, dim),
        )
        self.rearrange = Rearrange('b c f (t p2) -> b t (c f p2)', p2=patch_size[1]) # Rearrange layer
        self.pos_embedding = nn.Parameter(torch.randn(1, num_patches + 1, dim))
        self.cls_token = nn.Parameter(torch.randn(1, 1, dim))
        self.dropout = nn.Dropout(emb_dropout)

        self.transformer = Transformer(dim, depth, heads, dim_head=dim // heads, mlp_dim=mlp_dim, dropout=dropout)

        self.mlp_head = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, num_classes)
        )

    def forward(self, x):
         # Pad the time dimension if it's not divisible by patch_size
        b, c, f, t = x.shape
        pad_len = (self.patch_size[1] - (t % self.patch_size[1])) % self.patch_size[1]
        x = F.pad(x, (0, pad_len))  # Pad along time axis

        x = self.rearrange(x) # Rearrange after padding
        x = self.to_patch_embedding(x)

        b, n, _ = x.shape
        cls_tokens = repeat(self.cls_token, '() n d -> b n d', b=b)
        x = torch.cat((cls_tokens, x), dim=1)
        x = x + self.pos_embedding[:, :(n + 1)] # positional embedding scaled by the patch embedding
        x = self.dropout(x)

        x = self.transformer(x)
        x = x[:, 0]  # CLS token output
        return self.mlp_head(x)


# === Example Usage ===
model = KeywordTransformer(
    spectrogram_size=(40, 101),  # (Mel bands, time frames)
    patch_size=(40, 16),  # Only split along time
    num_classes=30,  # Based on label_dict
    dim=128,
    depth=6,
    heads=8,
    mlp_dim=256,
    dropout=0.1,
    emb_dropout=0.1
)

dummy_input = torch.randn(1, 1, 40, 101)  # (batch, channels, mel, time)
output = model(dummy_input)
print("Output Shape:", output.shape)  # Expected: (1, 30)

Output Shape: torch.Size([1, 30])


In [ ]:
cd /content/drive/MyDrive/Keyword_spotting_transformer/data/train_augmented_pinknoise

/content/drive/MyDrive/Keyword_spotting_transformer/data/train_augmented_pinknoise


In [ ]:
import numpy as np
import os

# Example path (adjust if needed)
sample_path = "/content/drive/MyDrive/Keyword_spotting_transformer/data/train_augmented_pinknoise/zero_xyz123.npy"

# Load and inspect
sample = np.load(sample_path)
print("Shape:", sample.shape)
print("Dtype:", sample.dtype)
print("Min value:", np.min(sample))
print("Max value:", np.max(sample))


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Keyword_spotting_transformer/data/train_augmented_pinknoise/zero_xyz123.npy'